# 基于隐马尔科夫模型的双序列比对

1. 基于多序列算法的论文公式改为双序列比对，选择种类较少的DNA序列，更为直观看出差异。
2. 发射概率和转移概率可根据具体情况修改
3. 使用Viterbi动态规划算法。 **这个过程中使用对数变换，避免由于连乘导致的数值下溢问题**
4. 最优路径输出规则：**seq1与seq2（i，j），若相同则输出'M'；若seq1的该碱基与seq2不同，要试探seq2的下一个碱基（i，j+1），则输出'D';若seq1的该碱基与seq2不同，要试探seq1的下一个碱基（i+1，j），则输出'I'** 起始位置（0,0）默认为匹配状态（M）开始
5. 可以自定义序列输出测试


## 算法原理
基于**多序列比对**的论文算法实现
下面简要介绍算法原理

首先定义一个长度为 N 的序列统计特征 P,它是一系列概率集合ei(b), ei(b)表示在位置 i 出现字符 b 的概率。在给定条件 P 下序列$\mathbf{X=(x1,x2,\cdots,xN)}$发生的概率为

$$P(X\mid P)=\prod_{i=1}e_i(x_i)$$

如果不考虑空位，则X与P的比对得分为：

$$Score(X\mid P)=\sum_{i=1}^N\log\frac{e_i(x_i)}{p(x_i)}$$

根据生物的进化功能，在进行多重比对时，除匹配 Mj 状态外， 我们要允许“插入”状态Ij和“删除”状态Dj，用于多序列比对的隐马尔可夫模型如下图所示：
![image.png](attachment:image.png)

使用 Viterbi 算法，将 X=(x1,x2,...,xL)与长度为 N 的统计特征 P 进行比对。假设有 n 个序列 X1,X2,···,Xn,如果 HMM 已知，在进行多重比对时，只需要将 Xj与 HMM 比对，再通过适当地插入空格可得到一个多重比对。

1. ${\mathrm{v}}_{\mathrm{j} }^{\mathrm{m} }($i)代表子序列$(\mathrm{x}_1,\mathrm{x}_2,\cdots,\mathrm{x}_{\mathrm{i}})$与 HMM 模型 P 的匹配概率对数记分值，该匹配以状态 M$_\mathrm{j}$释放字符 $x_\mathrm{i}$作为最后操作；

$$\nu_{j}^{M}(i)=\log\frac{e_{M_{j}}(x_{i})}{p(x_{i})}+\max\begin{cases}\nu_{j-1}^{M}(i-1)+\log f_{M_{j-1}M_{j}},\\\nu_{j-1}^{J}(i-1)+\log f_{I_{j-1}M_{j}},\\\nu_{j-1}^{D}(i-1)+\log f_{D_{j-1}M_{j};}\end{cases}$$

2. $v_{j}^{\mathrm{I} }$( i) 代表子序列$( x_1$, $x_2$, $\cdots$, $x_j)$与 HMM 模型 P 的匹配概率对数记分值，该匹配以状态 $I_j$释放字符 $x_i$作为最后操作；

$$\nu_{j}^{I}(i)=\log\frac{e_{I_{j}}(x_{i})}{p(x_{i})}+\max\begin{cases}\nu_{j}^{M}(i-1)+\log f_{M_{j}I_{j}},\\\nu_{j}^{I}(i-1)+\log f_{I_{j}I_{j}},\\\nu_{j}^{D}(i-1)+\log f_{D_{j}I_{j};}\end{cases}$$

3. $\mathbf{v} _{\mathrm{j} }^{\mathrm{D} }( \mathbf{i} )$代表子序列$(\mathbf{x}_1,\mathbf{x}_2,\cdots\mathbf{x}_{\mathrm{i}})$与 HMM 模型 P 的匹配概率对数记分值，该匹配以状态 $\mathcal{D}_\mathrm{j}$结束。

$$\nu_{j}^{D}(i)=\max\begin{cases}\nu_{j-1}^{M}(i)+\log f_{M_{j-1}D_{j}},\\\nu_{j-1}^{I}(i)+\log f_{I_{j-1}D_{j}},\\\nu_{j-1}^{D}(i)+\log f_{D_{j-1}D_{j}};\end{cases}$$

## 最优得分
则最优得分计算如下：

$$Score(X\mid\Pi^{\cdot})=\max\begin{cases}\nu_{N}^{M}\left(L\right)+\log f_{M_{N}end}\\\nu_{N}^{I}\left(L\right)+\log f_{I_{N}end}\\\nu_{N}^{D}\left(L\right)+\log f_{D_{N}end}\end{cases}.$$

## 最优路径
回溯得到的路径进行匹配展示

In [1]:
import numpy as np

# 转移概率
match_to_match = 0.9
match_to_insert = 0.05
match_to_delete = 0.05
insert_to_match = 0.8
insert_to_insert = 0.2
delete_to_match = 0.8
delete_to_delete = 0.2

# 发射概率
emit_prob = {
    'A': {'M': 0.25, 'I': 0.25, 'D': 0},
    'T': {'M': 0.25, 'I': 0.25, 'D': 0},
    'C': {'M': 0.25, 'I': 0.25, 'D': 0},
    'G': {'M': 0.25, 'I': 0.25, 'D': 0}
}

# 判断两个碱基是否匹配
def is_match(x, y):
    return (x == 'A' and y == 'A') or (x == 'T' and y == 'T') or (x == 'C' and y == 'C') or (x == 'G' and y == 'G')

# Viterbi算法（动态规划和回溯）
def viterbi(sequence1, sequence2):
    L1, L2 = len(sequence1), len(sequence2)
    
    # 初始化动态规划表和回溯指针表
    v_match = np.full((L1 + 1, L2 + 1), -np.inf)
    v_insert = np.full((L1 + 1, L2 + 1), -np.inf)
    v_delete = np.full((L1 + 1, L2 + 1), -np.inf)
    backtrace = np.full((L1 + 1, L2 + 1), None, dtype=object)
    
    # 边界条件
    v_match[0, 0] = 0

    for i in range(1, L1 + 1):
        for j in range(1, L2 + 1):
            x_i = sequence1[i - 1]
            y_j = sequence2[j - 1]
            
            # 计算匹配状态
            if is_match(x_i, y_j):  # 仅当字符匹配时计算
                match_score = emit_prob[x_i]['M'] * emit_prob[y_j]['M']
            else:
                match_score = 1e-10  # 如果不匹配，使用一个很小的值避免对数无效
            
            match_choices = [
                (v_match[i - 1, j - 1] + np.log(match_to_match * match_score), 'M'),
                (v_insert[i - 1, j - 1] + np.log(insert_to_match * match_score), 'I'),
                (v_delete[i - 1, j - 1] + np.log(delete_to_match * match_score), 'D')
            ]
            v_match[i, j], backtrace[i, j] = max(match_choices, key=lambda x: x[0])

            # 计算插入状态
            insert_score = emit_prob[x_i]['I']
            insert_choices = [
                (v_match[i - 1, j] + np.log(match_to_insert * insert_score), 'M'),
                (v_insert[i - 1, j] + np.log(insert_to_insert * insert_score), 'I')
            ]
            v_insert[i, j], _ = max(insert_choices, key=lambda x: x[0])

            # 计算删除状态
            delete_choices = [
                (v_match[i, j - 1] + np.log(match_to_delete), 'M'),
                (v_delete[i, j - 1] + np.log(delete_to_delete), 'D')
            ]
            v_delete[i, j], _ = max(delete_choices, key=lambda x: x[0])

    # 终止条件，选择得分最高的路径
    final_scores = [
        (v_match[L1, L2], 'M'),
        (v_insert[L1, L2], 'I'),
        (v_delete[L1, L2], 'D')
    ]
    final_score, final_state = max(final_scores, key=lambda x: x[0])

    # 回溯路径
    i, j = L1, L2
    optimal_path = []
    
    while i > 0 or j > 0:
        optimal_path.append((i, j, final_state))
        
        if final_state == 'M':
            prev_state = backtrace[i, j]
            i, j = i - 1, j - 1
        elif final_state == 'I':
            prev_state = 'M' if v_match[i - 1, j] + np.log(match_to_insert * emit_prob[sequence1[i - 1]]['I']) == v_insert[i, j] else 'I'
            i -= 1
        elif final_state == 'D':
            prev_state = 'M' if v_match[i, j - 1] + np.log(match_to_delete) == v_delete[i, j] else 'D'
            j -= 1

        final_state = prev_state

    optimal_path.reverse()  # 反转路径以从起点开始
    return final_score, optimal_path

In [2]:
# 测试示例
sequence1 = "ATCG"
sequence2 = "ATGC"
score, path = viterbi(sequence1, sequence2)
print("最优得分:", score)
print("最优路径:")
for step in path:
    print(step)

最优得分: -16.12938965757708
最优路径:
(1, 1, 'M')
(2, 2, 'M')
(2, 3, 'D')
(3, 4, 'M')
(4, 4, 'I')


In [3]:
sequence3 = "GACCGCGATTTGTCGGCCCCCAGCTAACGATCCG"
sequence4 = "GTCCGCGATTTGTCGGCCCCCAGCTAACGATCCG"
score1, path1 = viterbi(sequence3, sequence4)
print("最优得分:", score1)
print("最优路径:")
for step in path1:
    print(step)

最优得分: -111.22175457061417
最优路径:
(1, 1, 'M')
(1, 2, 'D')
(1, 3, 'D')
(1, 4, 'D')
(1, 5, 'D')
(1, 6, 'D')
(1, 7, 'D')
(2, 8, 'M')
(3, 8, 'I')
(4, 8, 'I')
(5, 8, 'I')
(6, 8, 'I')
(7, 8, 'I')
(8, 8, 'I')
(9, 9, 'M')
(10, 10, 'M')
(11, 11, 'M')
(12, 12, 'M')
(13, 13, 'M')
(14, 14, 'M')
(15, 15, 'M')
(16, 16, 'M')
(17, 17, 'M')
(18, 18, 'M')
(19, 19, 'M')
(20, 20, 'M')
(21, 21, 'M')
(22, 22, 'M')
(23, 23, 'M')
(24, 24, 'M')
(25, 25, 'M')
(26, 26, 'M')
(27, 27, 'M')
(28, 28, 'M')
(29, 29, 'M')
(30, 30, 'M')
(31, 31, 'M')
(32, 32, 'M')
(33, 33, 'M')
(34, 34, 'M')
